In [22]:
# https://github.com/pyg-team/pytorch_geometric/blob/master/examples/sgc.py
# SGC Model Using torch geometric with Cora
import torch
from torch_geometric.datasets import Planetoid # The citation network datasets "Cora", "CiteSeer" and "PubMed"
import os
from torch_geometric.nn import SGConv
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import torch.nn.functional as F

torch.cuda.is_available()

True

In [23]:
# DOWNLOAD DATA
path = os.getcwd()
dataset = Planetoid(path, "Cora")

In [24]:
# IMPORT DATA
data = dataset[0]
print('Coda: ', data)

Coda:  Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [25]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [26]:
# CONSTRUCT THE MODEL
sgc_model = SGConv( in_channels=data.num_features, # nr of features
                    out_channels=dataset.num_classes, # dimension of embedding
                    K = 1,
                    cached=True
)


In [27]:
# GET EMBEDDING
print("Shape of the original data: ", data.x.shape)
print("Shape of the embeddeding data: ", sgc_model(data.x, data.edge_index).shape)

Shape of the original data:  torch.Size([2708, 1433])
Shape of the embeddeding data:  torch.Size([2708, 7])


In [28]:
# CONSTRUCT THE MODEL FOR CLASSIFICATION
class SGCNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = SGConv(in_channels=data.num_features, # nr of features
                    out_channels=dataset.num_classes, # dimension of embedding
                    K = 1,
                    cached=True
        )

    def forward(self):
        x = self.conv1(data.x, data.edge_index) # applying convolution to data
        # computation of log softmax

        return F.log_softmax(x, dim=1) 

In [29]:
sgc_model, data = SGCNet().to(device), data.to(device)
optimizer = torch.optim.Adam(sgc_model.parameters(), lr=0.2, weight_decay=0.005)

# What are the learning parameters:
for i, parameter in sgc_model.named_parameters():
    print("Parameter {}".format(i))
    print("Shape: ", parameter.shape)

Parameter conv1.lin.weight
Shape:  torch.Size([7, 1433])
Parameter conv1.lin.bias
Shape:  torch.Size([7])


In [32]:
# TRAIN FUNCTION
def train():
    sgc_model.train() # set the model.training to be True
    optimizer.zero_grad() # Reset the gradient
    predicted_y = sgc_model() # predicted y in log softmax probability
    true_y = data.y # true labels
    losses = F.nll_loss(predicted_y[data.train_mask], true_y[data.train_mask])
    losses.backward()
    optimizer.step() # update the params such that is minimized the losses

# TEST FUNCTION
def test():
    sgc_model.eval() # set the model.training to be false
    logits = sgc_model() # log probability of all data
    accs = []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

In [33]:
# PUTTING IT ALL TOGETHER
best_val_acc = test_acc = 0
for epoch in range(1, 101):
    train()
    train_acc, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, '
          f'Val: {best_val_acc:.4f}, Test: {test_acc:.4f}')

Epoch: 001, Train: 0.9643, Val: 0.5600, Test: 0.5970
Epoch: 002, Train: 0.9929, Val: 0.7300, Test: 0.7580
Epoch: 003, Train: 0.9929, Val: 0.7300, Test: 0.7580
Epoch: 004, Train: 1.0000, Val: 0.7380, Test: 0.7510
Epoch: 005, Train: 0.9857, Val: 0.7380, Test: 0.7510
Epoch: 006, Train: 0.9857, Val: 0.7380, Test: 0.7510
Epoch: 007, Train: 1.0000, Val: 0.7380, Test: 0.7510
Epoch: 008, Train: 1.0000, Val: 0.7380, Test: 0.7510
Epoch: 009, Train: 1.0000, Val: 0.7380, Test: 0.7510
Epoch: 010, Train: 0.9929, Val: 0.7380, Test: 0.7510
Epoch: 011, Train: 0.9929, Val: 0.7440, Test: 0.7690
Epoch: 012, Train: 1.0000, Val: 0.7540, Test: 0.7730
Epoch: 013, Train: 1.0000, Val: 0.7680, Test: 0.7740
Epoch: 014, Train: 1.0000, Val: 0.7680, Test: 0.7740
Epoch: 015, Train: 1.0000, Val: 0.7680, Test: 0.7740
Epoch: 016, Train: 1.0000, Val: 0.7680, Test: 0.7740
Epoch: 017, Train: 1.0000, Val: 0.7680, Test: 0.7740
Epoch: 018, Train: 1.0000, Val: 0.7680, Test: 0.7740
Epoch: 019, Train: 1.0000, Val: 0.7680, Test: 